# Directory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Deep_learning/Deep\ Learning

/content/drive/MyDrive/Deep_learning/Deep Learning


# Libararies

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, IterableDataset

# calculate train time, writing train data to files etc.
import os
import sys
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from random import randint
from pathlib import Path
from scipy import signal
from scipy.signal import find_peaks
from scipy.interpolate import interp1d,UnivariateSpline
from sklearn.preprocessing import MinMaxScaler
from numpy import NaN, Inf, arange, isscalar, asarray, array

import pdb

# Dataset

In [18]:
class CoolDataset(IterableDataset):

    def __init__(self, dir_path: str, seq_length: str, input_size: str, samples_per_events: str, convolution=True):

        # Opens .csv file and read headers
        # Needs to decide what inputs to have and work on HS or FO

        super().__init__()

        self.files = tuple(Path(dir_path).glob("**/*.csv"))
        self.seq_length = seq_length
        self.input_size = input_size
        self.SAMPLES_PER_EVENT = samples_per_events
        self.window = signal.gaussian(8, std=3)
        self.convolution = convolution
        self.scaler = MinMaxScaler(feature_range=(0,1))

        assert seq_length % 2 == 0, "Please pass an even seq length"

    def __iter__(self):

        # Initialise Counter for events and files
        self.file_nr = 0
        self.event_in_file = 0
        self._sample_nr = 0

        return self

    def __next__(self):
        # Reads the current file and looks for event
        df, fname = self.read_file(self.files[self.file_nr])  # could be cached so you dont read it anew every iteration
        events = df[df['FO'] == 1]  # true where an event occurs, false everywhere else, use as mask

        if events.shape[0] > 0:
            if self._sample_nr < self.SAMPLES_PER_EVENT:
                # just give back the current event again, with different sampling, until we have generated
                # SAMPLES_PER_EVENT such samples

                event_frame = events.iloc[self.event_in_file].name
                indx_data, input_data, output_data = self.sample_seq_around_event_frame(df, event_frame)
                self._sample_nr += 1
            else:
                self.event_in_file += 1  # work on the next event in this file
                self._sample_nr = 0  # reset for the next event

                # check whether we are done with this file
                # otherwise we return the next event on the beginning of the next iteration
                if self.event_in_file >= len(events):
                    self.file_nr += 1
                    # If there still are files to run, it resets the variables
                    if self.file_nr < len(self.files):
                        logging.info("File is complete. Going to new file...")
                        self.event_in_file = 0
                        self._sample_nr = 0
                        return next(self)
                    else:
                        # processed the last file, we are done
                        logging.info("File is complete. All files done. Stopping...")
                        raise StopIteration
                elif self.event_in_file < len(events):
                    event_frame = events.iloc[self.event_in_file].name
                    indx_data, input_data, output_data = self.sample_seq_around_event_frame(df, event_frame)
                    self._sample_nr += 1
        else:
            logging.info("No events detected")
            self.file_nr += 1
            self.event_in_file = 0
            self._sample_nr = 0
            return next(self)

        return indx_data, input_data, output_data, fname

    def sample_seq_around_event_frame(self, df, event_idx):

        if event_idx >= 4:
            start_idx = event_idx - randint(4, self.seq_length / 7.5)
            if start_idx > 0:
                end_idx = start_idx + self.seq_length
                if end_idx <= len(df):
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']
                elif end_idx > len(df):
                    end_idx = len(df)
                    start_idx = end_idx - self.seq_length
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']
            elif start_idx <= 4:
                start_idx = event_idx
                end_idx = start_idx + self.seq_length
                indx = df.iloc[start_idx:end_idx]['ID']
                input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                output = df.iloc[start_idx:end_idx]['FO']
                if end_idx <= len(df):
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']
                elif end_idx > len(df):
                    end_idx = len(df)
                    start_idx = end_idx - self.seq_length
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']

        if self.convolution:
            output = signal.convolve(output, self.window, mode='same')
        input = self.scaler.fit_transform(input)
        indx = indx.to_numpy()
        
        assert input.shape[0] == output.shape[0] == self.seq_length
        return torch.tensor(indx),torch.tensor(input), torch.tensor(output)

    def read_file(self, f):
        df = pd.read_csv(open(f, "r"))
        fname = os.path.basename(f)
        if fname[0:2] == 'RT':
            df = df.drop([#'ID',
                          'class',
                          #'RTOE_X', 'RTOE_Y', 'RTOE_Z', 'V_RTOE_X', 'V_RTOE_Y', 'V_RTOE_Z',
                          #'RHLX_X', 'RHLX_Y', 'RHLX_Z', 'V_RHLX_X', 'V_RHLX_Y', 'V_RHLX_Z',
                          #'RHEE_X', 'RHEE_Y', 'RHEE_Z', 'V_RHEE_X', 'V_RHEE_Y', 'V_RHEE_Z',
                          'RPMT5_X', 'RPMT5_Y', 'RPMT5_Z', 'V_RPMT5_X', 'V_RPMT5_Y', 'V_RPMT5_Z'
                          ], axis=1)
        elif fname[0:2] == 'LT':
            df = df.drop([#'ID',
                          'class',
                          #'LTOE_X', 'LTOE_Y', 'LTOE_Z', 'V_LTOE_X', 'V_LTOE_Y', 'V_LTOE_Z',
                          #'LHLX_X', 'LHLX_Y', 'LHLX_Z', 'V_LHLX_X', 'V_LHLX_Y', 'V_LHLX_Z',
                          #'LHEE_X', 'LHEE_Y', 'LHEE_Z', 'V_LHEE_X', 'V_LHEE_Y', 'V_LHEE_Z',
                          'LPMT5_X', 'LPMT5_Y', 'LPMT5_Z', 'V_LPMT5_X', 'V_LPMT5_Y', 'V_LPMT5_Z',
                          ], axis=1)
        return df, fname

# Model

In [6]:
class Network(nn.Module):
    # TO DO
    def __init__(self, config):
        super(Network, self).__init__()

        # Model construct Configuration
        self.batch_size  = config.batch_size
        self.num_layers  = config.num_layers
        self.hidden_size = config.hidden_size
        self.device      = config.device

        self.lstm = nn.LSTM(config.input_size, self.hidden_size, self.num_layers, dropout=config.drop_out, batch_first=True,
                            bidirectional=True)
        self.linear = nn.Linear(self.hidden_size * 2, config.output_size, bias=True)
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, x):
        hidden, cell = self.init_hidden()
        out, _ = self.lstm(x, (hidden, cell))
        logits = self.linear(out)

        return logits[:, :, -1]

    def init_hidden(self):
        weight = next((self.parameters())).data
        hidden, cell = (weight.new(self.num_layers * 2, self.batch_size, self.hidden_size).zero_().to(self.device),
                        weight.new(self.num_layers * 2, self.batch_size, self.hidden_size).zero_().to(self.device))
        return hidden, cell

# Peak Detection

In [7]:
class peak_detection:

    def peak_comp(self, annotated, predicted, filename):
        dist = []

        if len(predicted) == 0 or len(annotated) == 0:
            print(f"{filename}_no events detection")
            return -1

        for a in predicted:
            dist = dist + [min(np.abs(annotated - a))]

        if not len(dist):
            print(f"{filename}")
            return -1

        return min(dist)

    def peakdetect(self, v, delta, x=None):
        maxtab = []
        mintab = []

        if x is None:
            x = arange(len(v))

        v = asarray(v)

        if len(v) != len(x):
            sys.exit('Input vectors v and x must have same length')

        if not isscalar(delta):
            sys.exit('Input argument delta must be a scalar')

        if delta <= 0:
            sys.exit('Input argument delta must be positive')

        mn, mx = Inf, -Inf
        mnpos, mxpos = NaN, NaN

        lookformax = True

        for i in arange(len(v)):
            this = v[i]
            if this > mx:
                mx = this
                mxpos = x[i]
            if this < mn:
                mn = this
                mnpos = x[i]

            if lookformax:
                if this < mx - delta:
                    maxtab.append((mxpos, mx))
                    mn = this
                    mnpos = x[i]
                    lookformax = False
            else:
                if this > mn + delta:
                    mintab.append((mnpos, mn))
                    mx = this
                    mxpos = x[i]
                    lookformax = True

        return array(maxtab), array(mintab)

    def eval_prediction(self, y_pred, y_true, filename, i, plot=True, shift=0):

        sdist = []

        # Interpolating the predicted signal
        y_pred_interp1d = interp1d(self.x, y_pred)
        y_pred_final = y_pred_interp1d(self.xnew)

        try:

            # peak_detection

            peakind_true,_ = find_peaks(y_true, height=0.5)
            peakind,_ = find_peaks(y_pred_final, height=0.5)

            if 'peakind' in locals():

                # peak_detection
                peakind_pred = [x for x in peakind]
                peakind_pred = self.xnew[peakind_pred]


            if any(peakind_pred):
                for k in peakind_pred:
                    if plot:
                        plt.axvline(x=k)

            sdist.append(self.peak_comp(self, peakind_true, [k + shift for k in peakind_pred], filename[:-4]))

            if plot:
                plt.plot(y_pred)
                plt.plot(y_true)
                plt.title(f"True vs Predicted {filename[:-4]} - {i}")
                axes = plt.gca()
                axes.set_xlim([0, y_true.shape[0]])
                my_file = f"{filename[:-4]} - {i}"
                plt.savefig(os.path.join(self.png_dir, my_file))
                plt.close()

        except:
            print(f"{filename} - No Detection, please check\n")
            edist = -1
            sdist.append(edist)

        return sdist

    def plot_stats(self, sdist):
        plt.title("Distance Error Histogram")
        plt.hist(sdist, 100, [0, 100])
        filtered = [k for k in sdist if k >= 0]

        def off_by(threshold, filtered):
            ob = [k for k in filtered if k <= threshold]
            nel = float(len(filtered))
            print("<= %d: %f" % (threshold, len(ob) / float(nel)))

        print("Error distribution:")
        off_by(1, filtered)
        off_by(3, filtered)
        off_by(5, filtered)
        off_by(10, filtered)
        off_by(60, filtered)
        print("Mean distance: %f" % (np.mean(filtered)))
        mean_distance = [np.mean(filtered)]
        plt.legend(mean_distance)
        plt.savefig(os.path.join(self.png_dir, f"{self.globaliter}_distance_error.png"))
        plt.close()
        return np.mean(filtered)

# Trainer

In [14]:
class Trainer:

    def __init__(self, model, config, name):

        # System configuration
        self.device = config.device

        # Model Construction
        self.model = Network(config).float()
        self.model.load_state_dict(model)
        self.model.to(self.device)
        print(self.model)

        # Peak detection and Evaluation
        self.eval_prediction = peak_detection.eval_prediction
        self.peak_comp = peak_detection.peak_comp
        self.peak_detect = peak_detection.peakdetect
        self.plot_stats = peak_detection.plot_stats
        self.x = np.linspace(0, 150, num=150, endpoint=True)
        self.xnew = np.linspace(0, 150, num=512, endpoint=True)
        self.weight = np.ones(150) * 12
        self.output_dir = config.output_dir
        self.png_dir = config.png_dir
        self.roc_dir = config.roc_dir
        self.globaliter = 0

        # DataLoader
        self.test_loader = DataLoader(
            CoolDataset(r'/content/drive/MyDrive/Deep_learning/Deep Learning/Data/iteration3/Test'f"/{name}", config.seq_length,config.input_size,config.samples_per_event, convolution=True), batch_size=config.batch_size, drop_last=True,
            shuffle=False)

    def test(self):
        self.model.eval()
        with torch.no_grad():
            for indx, data, target, filename in self.test_loader:

                error_dist = []
                filename_end = []
                TPR_temp_final = []
                TNR_temp_final = []
                data, target = data.to(self.device), target.to(self.device)

                predictions = self.model(data.float())
                pred = torch.sigmoid(predictions)

                correct_indx_positive = pred[target > 0.5]  # Should have batch_size * 1's
                correct_indx_negative = pred[target <= 0.5]  # Should have (batch_size*seq_length-batch_size) * 0'

                TPR_temp = len(correct_indx_positive[correct_indx_positive > 0.5]) / len( correct_indx_positive)
                TNR_temp = len(correct_indx_negative[correct_indx_negative <= 0.5]) / len( correct_indx_negative)

                TPR_temp = TPR_temp * 100
                TNR_temp = TNR_temp * 100
                # TPR_temp_final.append(TPR_temp)
                # TNR_temp_final.append(TNR_temp)
                
                for i in range(0, pred.shape[0]):

                    dist = (self.eval_prediction(self, pred[i], target[i], filename[i], i))
                    for j in range(0, len(dist)):
                        error_dist.append(dist[j])
                        filename_end.append(filename[i])

                dist_file  = np.column_stack((error_dist, filename_end))

                mean_dist  = self.plot_stats(self, error_dist)

                mean_name_dist  = "mean_dist"

                mean_dist_file  = np.column_stack((mean_dist, mean_name_dist))

                dist_file  = np.row_stack((dist_file, mean_dist_file))

                df_dist = pd.DataFrame(data=dist_file, )

                df_dist.columns  = ["Error in distance", "FileName"]

                df_dist.to_csv(os.path.join(self.output_dir, f"{self.globaliter}_error_file.csv"))

                # for i in range(0, predictions.shape[0]):
                #     indx_pred_target = np.transpose(np.row_stack((indx[i], pred[i], target[i])))
                #     df = pd.DataFrame(data=indx_pred_target, )
                #     df.columns = ["indx", "pred", "true_output"]
                #     df.to_csv(os.path.join(self.output_dir, f"{filename[i]}"))

                self.globaliter += 1
            return TPR_temp, TNR_temp

# Main

In [19]:
class Config:

    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

def main(loaded_model, model_config, name):
    trainer = Trainer(loaded_model, model_config, name)
    TPR_temp, TNR_temp = trainer.test()
    return TPR_temp, TNR_temp
if __name__ == '__main__':

    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Running on the GPU")
    else:
        device = torch.device("cpu")
        print("Running on the CPU")

    # Calling Models
    names    = ["ForeFoot","MidFoot","Heel"]

    model_dir = r'/content/drive/MyDrive/Deep_learning/Deep Learning/Results/LSTM/TO/3markers/HLXTOEHEE/v1/models'
    model_files = tuple(Path(f"{model_dir}/CheckPoint_DS/").glob("**/*pt"))

    for name in names:
      ROC_Final = []
      TPR_Final = []
      TNR_Final = []
      roc_dir       = f"{model_dir}/roc_TS/{name}/"
      if not os.path.exists(roc_dir):
          os.makedirs(roc_dir)

      for i in range(len(model_files)):
          model_name = os.path.basename((model_files[i]))
          loaded_model = torch.load(f"{model_dir}/CheckPoint_DS/" + model_name,map_location=torch.device(device))

          output_dir    = f"{model_dir}/evaluation/{name}/Results_final{i}/csv"
          png_dir       = f"{model_dir}/evaluation/{name}/Results_final{i}/png"
          print(model_name)

          # Directory Settings
          if not os.path.exists(output_dir):
              os.makedirs(output_dir)
              os.makedirs(png_dir)

          model_config = Config(

              # General Configuration
              device=device,
              batch_size=32,
              seq_length=150,
              samples_per_event = 1,
              output_size = 1,

              # LSTM
              input_size=18,
              hidden_size=512,
              num_layers=10,
              lr=0.001,
              drop_out=0.3,

              output_dir=output_dir,
              png_dir=png_dir,
              roc_dir = roc_dir
          )

          # ROC = 
          TPR,TNR = main(loaded_model, model_config, name)
          TPR_Final.append(TPR)
          TNR_Final.append(TNR)
      ROC = pd.DataFrame({f"TPR_{name}": TPR_Final,f"TNR_{name}":TNR_Final})
      ROC.to_csv(os.path.join(roc_dir, f"{name}_ROC.csv"),index=False)


Running on the CPU
TO-512-HS-10-NL-0.001-LR-0.3-DO-1-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 512, num_layers=10, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=1024, out_features=1, bias=True)
)
Error distribution:
<= 1: 0.093750
<= 3: 0.406250
<= 5: 0.750000
<= 10: 1.000000
<= 60: 1.000000
Mean distance: 3.482754
Error distribution:
<= 1: 0.218750
<= 3: 0.593750
<= 5: 1.000000
<= 10: 1.000000
<= 60: 1.000000
Mean distance: 2.404660
TO-512-HS-10-NL-0.001-LR-0.3-DO-2-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 512, num_layers=10, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=1024, out_features=1, bias=True)
)
Error distribution:
<= 1: 0.187500
<= 3: 0.437500
<= 5: 0.531250
<= 10: 0.656250
<= 60: 0.875000
Mean distance: 19.066659
Error distribution:
<= 1: 0.125000
<= 3: 0.343750
<= 5: 0.343750
<= 10: 0.656250
<= 60: 1.000000
Mean distance: 18.480797
TO-512-HS-10-NL-0.001-LR-0.3-DO-3-FScheckpoint.pt
Network(
  (lst